# Big Data Project on Flight delay Prediction

Team Members:
    Avinash
    Rajitha
    Sheron

About Data,Process

The Flight dataset has about records. With the help of the below segment of the code, we load and clean the Flight data by selecting the required columns & dropping unnecessary rows,build models as per requirement & finally use the models to predict flight delays.

Goal:
    The goal of the project is to predict Flight delays using past data & work with machine learning algorithms to find common patterns in late departures.

Why this is choosen for bigdata project?
Flight data is a massive data whcih is been collected from many years.Every hoour the data data keeps increasing at  a very high rate which is apt of bigdata project(the data grows huge in a short span)

Project Usefulness:
    This project will be used by airlines to act according to predicted dalyes thereby reducing business loss with increased customer satisfaction

##### Create a spark session and load the Data set

In [7]:
#importing Libraries
from pyspark.sql import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoderEstimator
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import IntegerType


In [8]:
spark = SparkSession.builder.appName('Flight').getOrCreate()

In [9]:
# File location and type
file_location = "/FileStore/tables/2019_Airlines.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c28
2019-01-01T00:00:00.000+0000,9E,3280,GNV,ATL,600,601,1.0,0.0,0,22.0,623,714,723,722,-1.0,0.0,null,0.0,83.0,81.0,51.0,300.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3281,MSP,CVG,1404,1359,-5.0,0.0,-1,15.0,1414,1629,1709,1633,-36.0,0.0,null,0.0,125.0,94.0,75.0,596.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3282,DTW,CVG,1220,1215,-5.0,0.0,-1,18.0,1233,1323,1345,1329,-16.0,0.0,null,0.0,85.0,74.0,50.0,229.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3283,TLH,ATL,1527,1521,-6.0,0.0,-1,14.0,1535,1618,1639,1625,-14.0,0.0,null,0.0,72.0,64.0,43.0,223.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3284,ATL,FSM,1902,1847,-15.0,0.0,-1,18.0,1905,1935,2005,1940,-25.0,0.0,null,0.0,123.0,113.0,90.0,579.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3285,DAY,MSP,900,853,-7.0,0.0,-1,25.0,918,948,1012,953,-19.0,0.0,null,0.0,132.0,120.0,90.0,574.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3286,JAN,ATL,1558,1553,-5.0,0.0,-1,33.0,1626,1818,1823,1832,9.0,0.0,null,0.0,85.0,99.0,52.0,341.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3287,LGA,CVG,1555,1551,-4.0,0.0,-1,31.0,1622,1816,1821,1824,3.0,0.0,null,0.0,146.0,153.0,114.0,585.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3288,JAX,LGA,1045,1037,-8.0,0.0,-1,17.0,1054,1235,1301,1239,-22.0,0.0,null,0.0,136.0,122.0,101.0,833.0,null,null,null,null,null,null
2019-01-01T00:00:00.000+0000,9E,3289,ATL,BMI,1245,1245,0.0,0.0,0,15.0,1300,1316,1332,1318,-14.0,0.0,null,0.0,107.0,93.0,76.0,533.0,null,null,null,null,null,null


In [10]:
file_location = "/FileStore/tables/carriers.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
carriers_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
carriers_df=carriers_df.withColumnRenamed("Code","OP_UNIQUE_CARRIER")
df=df.join(carriers_df,"OP_UNIQUE_CARRIER")

display(df)

OP_UNIQUE_CARRIER,FL_DATE,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c28,Description
9E,2019-01-01T00:00:00.000+0000,3280,GNV,ATL,600,601,1.0,0.0,0,22.0,623,714,723,722,-1.0,0.0,null,0.0,83.0,81.0,51.0,300.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3281,MSP,CVG,1404,1359,-5.0,0.0,-1,15.0,1414,1629,1709,1633,-36.0,0.0,null,0.0,125.0,94.0,75.0,596.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3282,DTW,CVG,1220,1215,-5.0,0.0,-1,18.0,1233,1323,1345,1329,-16.0,0.0,null,0.0,85.0,74.0,50.0,229.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3283,TLH,ATL,1527,1521,-6.0,0.0,-1,14.0,1535,1618,1639,1625,-14.0,0.0,null,0.0,72.0,64.0,43.0,223.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3284,ATL,FSM,1902,1847,-15.0,0.0,-1,18.0,1905,1935,2005,1940,-25.0,0.0,null,0.0,123.0,113.0,90.0,579.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3285,DAY,MSP,900,853,-7.0,0.0,-1,25.0,918,948,1012,953,-19.0,0.0,null,0.0,132.0,120.0,90.0,574.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3286,JAN,ATL,1558,1553,-5.0,0.0,-1,33.0,1626,1818,1823,1832,9.0,0.0,null,0.0,85.0,99.0,52.0,341.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3287,LGA,CVG,1555,1551,-4.0,0.0,-1,31.0,1622,1816,1821,1824,3.0,0.0,null,0.0,146.0,153.0,114.0,585.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3288,JAX,LGA,1045,1037,-8.0,0.0,-1,17.0,1054,1235,1301,1239,-22.0,0.0,null,0.0,136.0,122.0,101.0,833.0,null,null,null,null,null,null,Pinnacle Airlines Inc.
9E,2019-01-01T00:00:00.000+0000,3289,ATL,BMI,1245,1245,0.0,0.0,0,15.0,1300,1316,1332,1318,-14.0,0.0,null,0.0,107.0,93.0,76.0,533.0,null,null,null,null,null,null,Pinnacle Airlines Inc.


In [11]:
#checking number if rows & columns
print(df.count())
print(len(df.columns))

7422037
30

# Exploratory data analysis (EDA)

SQL Queries

In [14]:
data=df.select(['FL_DATE','OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST','CRS_DEP_TIME','DEP_TIME','DEP_DELAY','DEP_DEL15','DEP_DELAY_GROUP','TAXI_OUT','WHEELS_OFF','WHEELS_ON',
       'CRS_ARR_TIME','ARR_TIME','ARR_DELAY','CANCELLED','ACTUAL_ELAPSED_TIME','AIR_TIME','DISTANCE','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY',
        'Description'])

In [15]:
data=data.withColumn('fl_date_ts',to_timestamp(data['FL_DATE'],'yyyy-MM-dd')).\
      withColumn('fl_date',to_date(data['FL_DATE'],'yyyy-MM-dd'))

In [16]:
data=data.withColumn('Month',month(data['fl_date'])).\
      withColumn('DayofMonth',dayofmonth(data['fl_date'])).\
      withColumn('DayofWeek',date_format(data['fl_date'],'EEEE')).\
      withColumn('Monthname',date_format(data['fl_date'],'MMMMM'))

In [17]:
data = data.withColumn("dow", dayofweek("FL_DATE"))
data = data.withColumn("dayn", date_format("FL_DATE", 'EEE'))

In [18]:
data=data.dropna()


In [19]:
data.count()

Out[18]: 1389253

In [20]:
#converting into table to write sql queries
data.createOrReplaceTempView("IM")
spark.sql("describe IM").show(50)

+-------------------+---------+-------+
 col_name|data_type|comment|
+-------------------+---------+-------+
 fl_date| date| null|
 OP_UNIQUE_CARRIER| string| null|
 OP_CARRIER_FL_NUM| int| null|
 ORIGIN| string| null|
 DEST| string| null|
 CRS_DEP_TIME| int| null|
 DEP_TIME| int| null|
 DEP_DELAY| double| null|
 DEP_DEL15| double| null|
 DEP_DELAY_GROUP| int| null|
 TAXI_OUT| double| null|
 WHEELS_OFF| int| null|
 WHEELS_ON| int| null|
 CRS_ARR_TIME| int| null|
 ARR_TIME| int| null|
 ARR_DELAY| double| null|
 CANCELLED| double| null|
ACTUAL_ELAPSED_TIME| double| null|
 AIR_TIME| double| null|
 DISTANCE| double| null|
 CARRIER_DELAY| double| null|
 WEATHER_DELAY| double| null|
 NAS_DELAY| double| null|
 SECURITY_DELAY| double| null|
LATE_AIRCRAFT_DELAY| double| null|
 Description| string| null|
 fl_date_ts|timestamp| null|
 Month| int| null|
 DayofMonth| int| null|
 DayofWeek| string| null|
 Monthname| string| null|
 dow| int| null|
 dayn| string| null|
+-------------------+---------+-------+

In [21]:
flightnum_delay15=data.select("OP_CARRIER_FL_NUM","DEP_DEL15").groupby("OP_CARRIER_FL_NUM").agg(f.count('DEP_DEL15'),f.sum('DEP_DEL15'))\
.withColumnRenamed('count(DEP_DEL15)','scheduled')\
.withColumnRenamed('sum(DEP_DEL15)','Times_delayed')\
.withColumn('delay_rate %', f.round(col('Times_delayed')*100/col('scheduled'),2))\
.orderBy('Times_delayed', ascending=False)
display(flightnum_delay15)

OP_CARRIER_FL_NUM,scheduled,Times_delayed,delay_rate %
347,771,630.0,81.71
667,761,614.0,80.68
706,715,601.0,84.06
665,708,578.0,81.64
976,662,559.0,84.44
443,696,550.0,79.02
600,672,548.0,81.55
495,675,537.0,79.56
1202,642,537.0,83.64
478,669,534.0,79.82


In [22]:
carrier_delay15=data.select('OP_UNIQUE_CARRIER','Description','DEP_DEL15').groupBy('OP_UNIQUE_CARRIER','Description').agg(f.count("DEP_DEL15"),f.sum("DEP_DEL15"))\
.withColumnRenamed('count(DEP_DEL15)','scheduled')\
.withColumnRenamed('sum(DEP_DEL15)','Times_delayed')\
.withColumn('delay_rate %', f.round(col('Times_delayed')*100/col('scheduled'),2))\
.orderBy('delay_rate %', ascending=False)
display(carrier_delay15)

OP_UNIQUE_CARRIER,Description,scheduled,Times_delayed,delay_rate %
WN,Southwest Airlines Co.,233109,199076.0,85.4
B6,JetBlue Airways,74133,62269.0,84.0
F9,Frontier Airlines Inc.,33921,27512.0,81.11
9E,Pinnacle Airlines Inc.,44250,35699.0,80.68
YV,Mesa Airlines Inc.,46688,37585.0,80.5
OH,Comair Inc.,55727,44650.0,80.12
NK,Spirit Air Lines,37992,30201.0,79.49
G4,Allegiant Air,21541,17088.0,79.33
OO,Skywest Airlines Inc.,153115,120431.0,78.65
EV,Atlantic Southeast Airlines,33039,25501.0,77.18


In [23]:
airline_carriers=carrier_delay15.toPandas()['Description'].values.tolist()
percentage=carrier_delay15.toPandas()['delay_rate %'].values.tolist()

In [24]:
import matplotlib.pyplot as plt
plt.bar(airline_carriers,percentage,color='violet')
plt.xlabel('Airline carrier names')
plt.ylabel('Percentage Cancellations')
plt.show()
display()

In [25]:
delay_by_day=data.select('OP_UNIQUE_CARRIER','Description','DEP_DEL15','DayofWeek').groupBy('DayofWeek').agg(f.count("DEP_DEL15"),f.sum("DEP_DEL15"))\
.withColumnRenamed('count(DEP_DEL15)','scheduled')\
.withColumnRenamed('sum(DEP_DEL15)','Times_delayed')\
.withColumn('delay_rate %', f.round(col('Times_delayed')*100/col('scheduled'),2))\
.sort('delay_rate %',ascending=False)
display(delay_by_day)

DayofWeek,scheduled,Times_delayed,delay_rate %
Monday,214558,171134.0,79.76
Sunday,200078,157676.0,78.81
Friday,217081,170980.0,78.76
Thursday,223376,175316.0,78.48
Saturday,148694,115727.0,77.83
Tuesday,189117,145822.0,77.11
Wednesday,196349,150417.0,76.61


In [26]:
spark.sql("select (case when dayofweek in ('Monday','Tuesday','Wednesday','Thursday','Friday') then 'Weekdays'\
                        when dayofweek in ('Saturday','Sunday') then 'Weekend'\
                        else 'not identified' end) week_classif, sum(dep_delay) dep_delay_mins from IM\
          group by (case when dayofweek in ('Monday','Tuesday','Wednesday','Thursday','Friday') then 'Weekdays'\
               when dayofweek in ('Saturday','Sunday') then 'Weekend'\
		       else 'not identified' end)\
          order by dep_delay_mins desc").show()

+------------+--------------+
week_classif|dep_delay_mins|
+------------+--------------+
 Weekdays| 6.6701785E7|
 Weekend| 2.307139E7|
+------------+--------------+

In [27]:
spark.sql(" select DayofWeek, count(dep_delay) dep_delay_mins from IM\
            where dep_delay > 0 group by DayofWeek order by dep_delay_mins desc").show()

+---------+--------------+
DayofWeek|dep_delay_mins|
+---------+--------------+
 Thursday| 197512|
 Friday| 192489|
 Monday| 191182|
 Sunday| 177427|
Wednesday| 170666|
 Tuesday| 164573|
 Saturday| 130269|
+---------+--------------+

In [28]:
%sql
select dayn as Day,COUNT(DEP_DELAY) as DELAYS FROM IM where DEP_DELAY != 0.00 group by dayn order by DELAYS

Day,DELAYS
Sat,146244
Tue,185994
Wed,193049
Sun,196957
Mon,211357
Fri,213712
Thu,219854


In [29]:
%sql
select Delays, Day from (select  sum(DEP_DELAY) as Delays, dayn as Day,row_number() over (partition by dayn order by sum(DEP_DELAY) desc) as row_number from IM group by dayn order by dayn asc, Delays desc) as rows where row_number = 1

Delays,Day
1.3739094E7,Fri
1.4543504E7,Mon
9837179.0,Sat
1.3234211E7,Sun
1.447493E7,Thu
1.1767543E7,Tue
1.2176714E7,Wed


In [30]:
delay_by_Month=data.select('OP_UNIQUE_CARRIER','Monthname','DEP_DEL15').groupBy('Monthname').agg(f.count("DEP_DEL15"),f.sum("DEP_DEL15"))\
.withColumnRenamed('count(DEP_DEL15)','scheduled')\
.withColumnRenamed('sum(DEP_DEL15)','Times_delayed')\
.withColumn('delay_rate %', f.round(col('Times_delayed')*100/col('scheduled'),2))\
.sort('delay_rate %',ascending=False)
display(delay_by_Month)

Monthname,scheduled,Times_delayed,delay_rate %
July,135630,110238.0,81.28
June,153358,123700.0,80.66
April,108518,87190.0,80.35
August,133278,106710.0,80.07
May,126283,100639.0,79.69
December,126945,99444.0,78.34
March,106277,82699.0,77.81
September,83568,64078.0,76.68
October,105046,79898.0,76.06
February,119477,89980.0,75.31


In [31]:
spark.sql(" select monthname, sum(dep_delay) dep_delay_mins from IM group by monthname order by monthname desc").show()

+---------+--------------+
monthname|dep_delay_mins|
+---------+--------------+
September| 5192719.0|
 October| 6075453.0|
 November| 4994534.0|
 May| 8303159.0|
 March| 6565458.0|
 June| 1.0353112E7|
 July| 9862139.0|
 January| 6529791.0|
 February| 7490877.0|
 December| 8095680.0|
 August| 8989554.0|
 April| 7320699.0|
+---------+--------------+

In [32]:
%sql
select Delays, Month from (select sum(DEP_DELAY) as Delays, Month,row_number() over (partition by Month order by sum(DEP_DELAY) desc) as row_number from IM group by Month order by Month asc, Delays desc)

Delays,Month
6529791.0,1
7490877.0,2
6565458.0,3
7320699.0,4
8303159.0,5
1.0353112E7,6
9862139.0,7
8989554.0,8
5192719.0,9
6075453.0,10


In [33]:
dep_delay_range = data.withColumn(
    'DEP_DELAY_GROUP_RANGE',
    f.when(f.col("DEP_DELAY_GROUP")== 0, '0-15')\
     .when(f.col("DEP_DELAY_GROUP")== 1, '15-30')\
     .when(f.col("DEP_DELAY_GROUP")== 2, '30-45')\
     .when(f.col("DEP_DELAY_GROUP")== 3, '45-60')\
     .when(f.col("DEP_DELAY_GROUP")== 4, '60-75')\
     .when(f.col("DEP_DELAY_GROUP")== 5, '75-90')\
     .when(f.col("DEP_DELAY_GROUP")== 6, '90-105')\
     .when(f.col("DEP_DELAY_GROUP")== 7, '105-120')\
     .when(f.col("DEP_DELAY_GROUP")== 8, '120-135')\
     .when(f.col("DEP_DELAY_GROUP")== 9, '135-150')\
     .when(f.col("DEP_DELAY_GROUP")== 10, '150-165')\
     .when(f.col("DEP_DELAY_GROUP")== 11, '165-180')\
     .when(f.col("DEP_DELAY_GROUP")== 12, '180-195')\
     .when(f.col("DEP_DELAY_GROUP")== -1, '0-(-15)')\
     .when(f.col("DEP_DELAY_GROUP")== -2, '(-15)-(-30)')\
    .otherwise('On time')
)
display(dep_delay_range)

fl_date,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Description,fl_date_ts,Month,DayofMonth,DayofWeek,Monthname,dow,dayn,DEP_DELAY_GROUP_RANGE
2019-01-01,9E,3291,DTW,DAY,1540,1552,12.0,0.0,0,68.0,1700,1735,1654,1738,44.0,0.0,106.0,35.0,166.0,12.0,0.0,32.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-15
2019-01-01,9E,3295,DTW,EWR,1230,1353,83.0,1.0,5,18.0,1411,1516,1422,1521,59.0,0.0,88.0,65.0,488.0,0.0,0.0,59.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,75-90
2019-01-01,9E,3300,ATL,OAJ,2220,2216,-4.0,0.0,-1,56.0,2312,7,2347,16,29.0,0.0,120.0,55.0,399.0,0.0,0.0,29.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-(-15)
2019-01-01,9E,3317,FSM,ATL,1558,1550,-8.0,0.0,-1,10.0,1600,1817,1842,1859,17.0,0.0,129.0,77.0,579.0,0.0,0.0,17.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-(-15)
2019-01-01,9E,3322,DTW,LAN,2040,2036,-4.0,0.0,-1,88.0,2204,2229,2140,2233,53.0,0.0,117.0,25.0,74.0,0.0,0.0,53.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-(-15)
2019-01-01,9E,3328,CVG,MSP,700,817,77.0,1.0,5,10.0,827,856,810,900,50.0,0.0,103.0,89.0,596.0,50.0,0.0,0.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,75-90
2019-01-01,9E,3338,MGM,ATL,1140,1411,151.0,1.0,10,9.0,1420,1554,1338,1558,140.0,0.0,47.0,34.0,147.0,138.0,0.0,0.0,0.0,2.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,150-165
2019-01-01,9E,3340,DTW,JFK,2010,2006,-4.0,0.0,-1,63.0,2109,2217,2205,2224,19.0,0.0,138.0,68.0,509.0,0.0,0.0,19.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-(-15)
2019-01-01,9E,3374,MSP,CVG,1001,1100,59.0,1.0,3,13.0,1113,1334,1311,1339,28.0,0.0,99.0,81.0,596.0,28.0,0.0,0.0,0.0,0.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,45-60
2019-01-01,9E,3388,DTW,GRB,2015,2023,8.0,0.0,0,84.0,2147,2144,2046,2149,63.0,0.0,146.0,57.0,287.0,0.0,0.0,55.0,0.0,8.0,Pinnacle Airlines Inc.,2019-01-01T00:00:00.000+0000,1,1,Tuesday,January,3,Tue,0-15


In [34]:
display(dep_delay_range.select('OP_UNIQUE_CARRIER','Description','DEP_DELAY_GROUP_RANGE').groupby('DEP_DELAY_GROUP_RANGE').count().sort('count',ascending=False))

DEP_DELAY_GROUP_RANGE,count
30-45,222423
15-30,221868
0-15,159132
45-60,156397
0-(-15),142425
60-75,108273
180-195,91218
75-90,78540
90-105,59253
105-120,45872


In [35]:
spark.sql(" select DEST, avg(dep_delay) avg_dep_delay from IM group by DEST order by avg(dep_delay) desc").show(5)

+----+------------------+
DEST| avg_dep_delay|
+----+------------------+
 DVL|145.54310344827587|
 OGS|141.38095238095238|
 JMS|137.77300613496934|
 EKO|135.18421052631578|
 MQT| 132.9553264604811|
+----+------------------+
only showing top 5 rows

In [36]:
spark.sql(" select ORIGIN, avg(dep_delay) avg_dep_delay from IM group by ORIGIN order by avg(dep_delay) desc").show(5)

+------+------------------+
ORIGIN| avg_dep_delay|
+------+------------------+
 WYS|228.52380952380952|
 JMS|167.82954545454547|
 SHD|158.24590163934425|
 EKO|149.24242424242425|
 HYS|149.05652173913043|
+------+------------------+
only showing top 5 rows

In [37]:
delay_origin=dep_delay_range.groupBy('ORIGIN','DEST').agg(f.count("DEP_DEL15"),f.sum("DEP_DEL15"))\
.withColumnRenamed('count(DEP_DEL15)','scheduled')\
.withColumnRenamed('sum(DEP_DEL15)','Times_delayed')\
.withColumn('delay_rate %', f.round(col('Times_delayed')*100/col('scheduled'),2))\
.sort(['delay_rate %'],ascending=[False])
display(delay_origin)

ORIGIN,DEST,scheduled,Times_delayed,delay_rate %
GSP,BWI,7,7.0,100.0
STL,CVG,25,25.0,100.0
CLE,SLC,5,5.0,100.0
EGE,LGA,5,5.0,100.0
HTS,PGD,6,6.0,100.0
ABQ,SAT,1,1.0,100.0
ISP,MYR,13,13.0,100.0
OAK,EUG,22,22.0,100.0
ANC,BLI,2,2.0,100.0
SAN,EUG,5,5.0,100.0


In [38]:
spark.sql(" select Dest, count(dep_delay) dep_delay_mins from IM\
            where dep_delay > 0 group by Dest order by dep_delay_mins desc").show()

+----+--------------+
Dest|dep_delay_mins|
+----+--------------+
 ORD| 63461|
 DFW| 51783|
 ATL| 47885|
 DEN| 41479|
 LGA| 39156|
 SFO| 37921|
 EWR| 36086|
 LAX| 33757|
 CLT| 32648|
 BOS| 32354|
 IAH| 28366|
 LAS| 27263|
 PHX| 26648|
 MCO| 25041|
 DCA| 22508|
 JFK| 22227|
 SEA| 22064|
 DTW| 21830|
 MSP| 21340|
 PHL| 19779|
+----+--------------+
only showing top 20 rows

In [39]:
spark.sql(" select ORIGIN, count(dep_delay) dep_delay_mins from IM\
            where dep_delay > 0 group by ORIGIN order by dep_delay_mins desc").show(5)

+------+--------------+
ORIGIN|dep_delay_mins|
+------+--------------+
 ORD| 72009|
 DFW| 63287|
 ATL| 55599|
 DEN| 50874|
 CLT| 43282|
+------+--------------+
only showing top 5 rows

In [40]:
spark.sql(" select origin, dest, sum(dep_delay) dep_delay_in_mins from IM group by origin, dest order by dep_delay_in_mins desc").show(5)

+------+----+-----------------+
origin|dest|dep_delay_in_mins|
+------+----+-----------------+
 ORD| LGA| 309009.0|
 LGA| ORD| 306193.0|
 LAX| SFO| 219884.0|
 MCO| EWR| 207699.0|
 BOS| LGA| 207429.0|
+------+----+-----------------+
only showing top 5 rows

In [41]:
%sql
select ORIGIN, Delays, Month from (select ORIGIN, sum(DEP_DELAY) as Delays, Month,row_number() over (partition by Month order by sum(DEP_DELAY) desc) as row_number from IM group by Month,ORIGIN order by Month asc, Delays desc) as rows WHERE row_number = 1

ORIGIN,Delays,Month
ORD,501394.0,1
ORD,443311.0,2
DFW,331920.0,3
ATL,353140.0,4
DFW,512876.0,5
ORD,625782.0,6
ORD,649880.0,7
ORD,496673.0,8
ORD,396615.0,9
DFW,369294.0,10


In [42]:
dep_delay_range.groupby('WEATHER_DELAY','DEP_DEL15','DEP_DELAY_GROUP_RANGE').agg(f.count('DEP_DEL15'),f.sum('DEP_DEL15')).\
withColumnRenamed('count(DEP_DEL15)','scheduled').\
withColumnRenamed('sum(DEP_DEL15)','Times_delayed').\
sort('Times_delayed',ascending=False).show()

+-------------+---------+---------------------+---------+-------------+
WEATHER_DELAY|DEP_DEL15|DEP_DELAY_GROUP_RANGE|scheduled|Times_delayed|
+-------------+---------+---------------------+---------+-------------+
 0.0| 1.0| 30-45| 210764| 210764.0|
 0.0| 1.0| 15-30| 209794| 209794.0|
 0.0| 1.0| 45-60| 147287| 147287.0|
 0.0| 1.0| 60-75| 101312| 101312.0|
 0.0| 1.0| 180-195| 79877| 79877.0|
 0.0| 1.0| 75-90| 73039| 73039.0|
 0.0| 1.0| 90-105| 54789| 54789.0|
 0.0| 1.0| 105-120| 42234| 42234.0|
 0.0| 1.0| 120-135| 32042| 32042.0|
 0.0| 1.0| 135-150| 25233| 25233.0|
 0.0| 1.0| 150-165| 20405| 20405.0|
 0.0| 1.0| 165-180| 16031| 16031.0|
 15.0| 1.0| 15-30| 935| 935.0|
 16.0| 1.0| 15-30| 819| 819.0|
 17.0| 1.0| 15-30| 759| 759.0|
 18.0| 1.0| 15-30| 677| 677.0|
 19.0| 1.0| 15-30| 663| 663.0|
 20.0| 1.0| 15-30| 643| 643.0|
 21.0| 1.0| 15-30| 578| 578.0|
 22.0| 1.0| 15-30| 500| 500.0|
+-------------+---------+---------------------+---------+-------------+
only showing top 20 rows

In [43]:
spark.sql("select year(fl_date) year, (sum(CARRIER_DELAY)/sum(dep_delay))*100 carrier_delay_percent, (sum(WEATHER_DELAY)/sum(dep_delay))*100 weather_delay_percent,(sum(NAS_DELAY)/sum(dep_delay))*100 nas_delay_percent,(sum(SECURITY_DELAY)/sum(dep_delay))*100 security_delay_percent,(sum(LATE_AIRCRAFT_DELAY)/sum(dep_delay))*100 late_aircraft_delay_percent from IM group by year(fl_date)").show()

+----+---------------------+---------------------+------------------+----------------------+---------------------------+
year|carrier_delay_percent|weather_delay_percent| nas_delay_percent|security_delay_percent|late_aircraft_delay_percent|
+----+---------------------+---------------------+------------------+----------------------+---------------------------+
2019| 32.69680503112428| 5.884275564499084|25.670089088416447| 0.14869029640535716| 42.413473735333525|
+----+---------------------+---------------------+------------------+----------------------+---------------------------+

In [44]:
spark.sql("select (case when dep_delay > '0' then 'Delayed'\
                        else 'NotDelayed' end) Timings, count(dep_delay) count from IM\
           group by (case when dep_delay > '0' then 'Delayed'\
                        else 'NotDelayed' end)\
          order by count(dep_delay) desc").show()

+----------+-------+
 Timings| count|
+----------+-------+
 Delayed|1224118|
NotDelayed| 165135|
+----------+-------+

In [45]:
Observation:

Highest delay is for flight number : 347
Highest delay is for carrier : Southwest Airlines Co.
Highest delays are from origin with destination : Orlando airport - LaGaurdia airport
Most of the delays are observed on Monday & July month
Based on defined delay ranges we observe that most of the delays falls under 30-45 minutes. 
Most of the delayes are caused by late aircraft delays & least are caused by security delays

##### Data pre-processing

In [47]:
data1=data.drop('fl_date','OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST','Month','Monthname','fl_date_ts')

In [48]:
#checking correlation with target variable-just for analysis
import six
for i in data1.columns:
    if not( isinstance(data1.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to target for ", i, data1.stat.corr('DEP_DELAY',i))

Correlation to target for CRS_DEP_TIME 0.037314874157115
Correlation to target for DEP_TIME 0.04840171185697959
Correlation to target for DEP_DELAY 1.0
Correlation to target for DEP_DEL15 0.35870417496028634
Correlation to target for DEP_DELAY_GROUP 0.7939718723218242
Correlation to target for TAXI_OUT -0.13188212125769713
Correlation to target for WHEELS_OFF 0.02310866453047953
Correlation to target for WHEELS_ON -0.0710863513309918
Correlation to target for CRS_ARR_TIME 0.0257633543800512
Correlation to target for ARR_TIME -0.07354034460053754
Correlation to target for ARR_DELAY 0.9744124231813703
Correlation to target for CANCELLED nan
Correlation to target for ACTUAL_ELAPSED_TIME -0.0817070785955507
Correlation to target for AIR_TIME -0.047429488618653384
Correlation to target for DISTANCE -0.023037227147542253
Correlation to target for CARRIER_DELAY 0.6449399805984256
Correlation to target for WEATHER_DELAY 0.3058926275793774
Correlation to target for NAS_DELAY 0.20117911394945703
Correlation to target for SECURITY_DELAY 0.023586502655598997
Correlation to target for LATE_AIRCRAFT_DELAY 0.5088408454774137
Correlation to target for DayofMonth 0.003111565717633
Correlation to target for dow -0.006644385076842909

In [49]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to make predictions

data2=dep_delay_range.select(['OP_UNIQUE_CARRIER','ORIGIN','DEST',
 'DEP_DELAY','DEP_DELAY_GROUP',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY'])
# Create a 70-30 train test split

train_data,test_data=data2.randomSplit([0.7,0.3])
#this shows how many rows are dropped


In [50]:
data2.printSchema()

# Model Building

Model1:K means Clustering

Goal is to predict the range in which most of the departures are falling

In [54]:
df1=dep_delay_range.select(['OP_UNIQUE_CARRIER','ORIGIN','DEST',
 'DEP_DELAY','DEP_DELAY_GROUP',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY'])

In [55]:
df1 = df1.withColumn("DEP_DELAY_GRP",df1["DEP_DELAY_GROUP"].cast(IntegerType()))

In [56]:
#To filter only the records with positive departure delays 
df1 = df1.filter("DEP_DELAY_GRP > 0")

In [57]:
# Use StringIndexer to convert the categorical columns to hold numerical data
OP_UNIQUE_CARRIER_indexer = StringIndexer(inputCol='OP_UNIQUE_CARRIER',outputCol='OP_UNIQUE_CARRIER_index',handleInvalid='keep')
ORIGIN_indexer = StringIndexer(inputCol='ORIGIN',outputCol='ORIGIN_index',handleInvalid='keep')
DEST_indexer = StringIndexer(inputCol='DEST',outputCol='DEST_index',handleInvalid='keep')
DEP_DELAY_indexer = StringIndexer(inputCol='DEP_DELAY',outputCol='DEP_DELAY_index',handleInvalid='keep')
CARRIER_DELAY_indexer = StringIndexer(inputCol='CARRIER_DELAY',outputCol='CARRIER_DELAY_index',handleInvalid='keep')
WEATHER_DELAY_indexer = StringIndexer(inputCol='WEATHER_DELAY',outputCol='WEATHER_DELAY_index',handleInvalid='keep')
NAS_DELAY_indexer = StringIndexer(inputCol='NAS_DELAY',outputCol='NAS_DELAY_index',handleInvalid='keep')
SECURITY_DELAY_indexer = StringIndexer(inputCol='SECURITY_DELAY',outputCol='SECURITY_DELAY_index',handleInvalid='keep')
LATE_AIRCRAFT_DELAY_indexer = StringIndexer(inputCol='LATE_AIRCRAFT_DELAY',outputCol='LATE_AIRCRAFT_DELAY_index',handleInvalid='keep')

In [58]:
# Vector assembler is used to create a vector of input features
assembler = VectorAssembler(inputCols=['OP_UNIQUE_CARRIER_index','ORIGIN_index','DEST_index',
                                       'DEP_DELAY_index','CARRIER_DELAY_index','WEATHER_DELAY_index',
                                       'NAS_DELAY_index','SECURITY_DELAY_index','LATE_AIRCRAFT_DELAY_index'],
                            outputCol="features")

In [59]:
pipe = Pipeline(stages=[OP_UNIQUE_CARRIER_indexer,ORIGIN_indexer,DEST_indexer,DEP_DELAY_indexer,
                        CARRIER_DELAY_indexer,WEATHER_DELAY_indexer,NAS_DELAY_indexer,SECURITY_DELAY_indexer,LATE_AIRCRAFT_DELAY_indexer,assembler])

This will b unsupervised learning so we dont use train-test split. So,we dont feed & transform instead transform directly

In [61]:
final_data=pipe.fit(df1).transform(df1) 

In [62]:
kmeans_model = KMeans(k=12)

The number 12 is the number of clusters to divide the data into. As the number is larger than 2,it causes this value ClusteringEvaluator() to fall below 0.5, meaning it’s not a clear division. Another way to check the optimal number of clusters would be to plot an elbow curve.

In [64]:
fit_model = kmeans_model.fit(final_data)

In [65]:
wssse = fit_model.computeCost(final_data)

The below code is for sum of squared errors and it computes the distance of each data point from its guess as to the center of the cluster, adjusts the guesses, then repeats until the number reaches its minimum. The distance of each point from this central point is squared so that distance is always positive. The goal is to have the smallest number possible—the shortest distance between all the data points.

In [67]:
print("The within set sum of squared error of the mode is {}".format(wssse))

The within set sum of squared error of the mode is 6911107329.709544

the algorithm shows the squared Euclidean distance.

If this number is negative, the data cannot be separated at all. Values closer to 1 indicate maximum separation. Values close to zero mean the data could barely be separated.

Ideally,we divide the data into two clusters, which explains why we have two sets of coordinates. Those are the centroids, or the central point of each cluster.If we try to divide it into a higher number of clusters, that becomes less accurate, as the data is not spaced sufficiently apart

In [70]:
centers = fit_model.clusterCenters()

In [71]:
results = fit_model.transform(final_data)

Evaluate Model

In [73]:
# Evaluate clustering by computing Silhouette score
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(results)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.3960986784832881

Model can be evaluated using silhouette_score,where high coefficient indicates that model perfomes well
0.39 is not bad.

In [75]:
results.groupby('prediction').count().sort('prediction').show()

+----------+------+
prediction| count|
+----------+------+
 0|511273|
 1| 3128|
 2| 2474|
 3| 27245|
 4| 77614|
 5| 47937|
 6| 5464|
 7|138086|
 8| 71209|
 9|147188|
 10| 32495|
 11| 22959|
+----------+------+

Above indicates the count with respect to each cluster.As this approach works with data that divides according to some common characteristics,we can mentioned the count shows how much data has common characteristics with respect to each cluster.

Model2:Logistic Regression

The goal is to build a predictive binary logistic regression model whether the flight is delayed by 15min or not

In [79]:
df2=dep_delay_range.select(['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','AIR_TIME','DISTANCE','DEST','DEP_DELAY','DEP_DEL15','ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY',
 'SECURITY_DELAY','LATE_AIRCRAFT_DELAY'])

In [80]:
df2 = df2.withColumn("DEP_DELAY15",df2["DEP_DEL15"].cast(IntegerType()))

In [81]:
train_data,test_data=df2.randomSplit([0.7,0.3])

In [82]:
# Use StringIndexer to convert the categorical columns to hold numerical data
OP_UNIQUE_CARRIER_indexer = StringIndexer(inputCol='OP_UNIQUE_CARRIER',outputCol='OP_UNIQUE_CARRIER_index',handleInvalid='keep')
OP_CARRIER_FL_NUM_indexer = StringIndexer(inputCol='OP_CARRIER_FL_NUM',outputCol='OP_CARRIER_FL_NUM_index',handleInvalid='keep')
ORIGIN_indexer = StringIndexer(inputCol='ORIGIN',outputCol='ORIGIN_index',handleInvalid='keep')
AIR_TIME_indexer = StringIndexer(inputCol='AIR_TIME',outputCol='AIR_TIME_index',handleInvalid='keep')
DEST_indexer = StringIndexer(inputCol='DEST',outputCol='DEST_index',handleInvalid='keep')
DEP_DELAY_indexer = StringIndexer(inputCol='DEP_DELAY',outputCol='DEP_DELAY_index',handleInvalid='keep')
CARRIER_DELAY_indexer = StringIndexer(inputCol='CARRIER_DELAY',outputCol='CARRIER_DELAY_index',handleInvalid='keep')
WEATHER_DELAY_indexer = StringIndexer(inputCol='WEATHER_DELAY',outputCol='WEATHER_DELAY_index',handleInvalid='keep')
NAS_DELAY_indexer = StringIndexer(inputCol='NAS_DELAY',outputCol='NAS_DELAY_index',handleInvalid='keep')
SECURITY_DELAY_indexer = StringIndexer(inputCol='SECURITY_DELAY',outputCol='SECURITY_DELAY_index',handleInvalid='keep')
LATE_AIRCRAFT_DELAY_indexer = StringIndexer(inputCol='LATE_AIRCRAFT_DELAY',outputCol='LATE_AIRCRAFT_DELAY_index',handleInvalid='keep')

In [83]:
# OneHotEncoderEstimator converts the indexed data into a vector which will be effectively handled by Logistic Regression model
data_encoder = OneHotEncoderEstimator(inputCols=['OP_UNIQUE_CARRIER_index','OP_CARRIER_FL_NUM_index','ORIGIN_index','AIR_TIME_index','DEST_index','DEP_DELAY_index',
                                                 'CARRIER_DELAY_index','WEATHER_DELAY_index','NAS_DELAY_index','SECURITY_DELAY_index','LATE_AIRCRAFT_DELAY_index'],
                                      outputCols=['OP_UNIQUE_CARRIER_vec','OP_CARRIER_FL_NUM_vec','ORIGIN_vec','AIR_TIME_vec','DEST_vec','DEP_DELAY_vec','CARRIER_DELAY_vec',
                                                  'WEATHER_DELAY_vec','NAS_DELAY_vec','SECURITY_DELAY_vec','LATE_AIRCRAFT_DELAY_vec'],
                                      handleInvalid='keep')

In [84]:
# Vector assembler is used to create a vector of input features
assembler = VectorAssembler(inputCols=['OP_UNIQUE_CARRIER_vec','OP_CARRIER_FL_NUM_vec','ORIGIN_vec','AIR_TIME_vec','DEST_vec','DEP_DELAY_vec','CARRIER_DELAY_vec',
                                                  'WEATHER_DELAY_vec','NAS_DELAY_vec','SECURITY_DELAY_vec','LATE_AIRCRAFT_DELAY_vec'],
                            outputCol="features")

In [85]:
lr_model = LogisticRegression(labelCol='DEP_DELAY15')

In [86]:
pipe = Pipeline(stages=[OP_UNIQUE_CARRIER_indexer,OP_CARRIER_FL_NUM_indexer,ORIGIN_indexer,AIR_TIME_indexer,DEST_indexer,DEP_DELAY_indexer,
                        CARRIER_DELAY_indexer,WEATHER_DELAY_indexer,NAS_DELAY_indexer,SECURITY_DELAY_indexer,LATE_AIRCRAFT_DELAY_indexer,data_encoder,assembler,lr_model])

In [87]:
fit_model=pipe.fit(train_data)

In [88]:
results = fit_model.transform(test_data)

In [89]:
results.select(['DEP_DELAY15','prediction']).show(n=1000)

+-----------+----------+
DEP_DELAY15|prediction|
+-----------+----------+
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 1.0|
 1

Model Evaluation:

In [91]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [92]:
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="DEP_DELAY15", predictionCol="prediction", metricName="accuracy")

In [93]:
accuracy = ACC_evaluator.evaluate(results)

In [94]:
print("The accuracy of the decision tree classifier is {}".format(accuracy))

The accuracy of the decision tree classifier is 0.9999640121207177

Accuracy doesnt add any weightage if the baseline is unknown.Accuracy has to be more than baseline Rare cases are there in input,so if output is all 0 then still the accuracy will be around 99.9% Always accuracy should be looked at data distribution perspective,so we should consider how many rare events are getting predictable. Accordingly decide to consider Accuracy as metric for model prediction.

Suggestion:Replicate the number of rare events in data so that the data will have a good combination of rare & common events

In [96]:
from sklearn.metrics import confusion_matrix

In [97]:
y_true = results.select("DEP_DELAY15")
y_true = y_true.toPandas()

y_pred = results.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
print("Below is the confusion matrix \n {}".format(cnf_matrix))

Below is the confusion matrix 
 [[ 90564 3]
 [ 12 326228]]

In [98]:
from pyspark.ml.regression import LinearRegression

In [99]:
df3=dep_delay_range.select(['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST','DEP_DELAY','DEP_DELAY_GROUP','DEP_DEL15','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY',
 'SECURITY_DELAY','LATE_AIRCRAFT_DELAY'])

In [100]:
df3 = df3.withColumn("DEP_DEL",df3["DEP_DELAY"].cast(IntegerType()))

In [101]:
# Use StringIndexer to convert the categorical columns to hold numerical data
OP_UNIQUE_CARRIER_indexer = StringIndexer(inputCol='OP_UNIQUE_CARRIER',outputCol='OP_UNIQUE_CARRIER_index',handleInvalid='keep')
OP_CARRIER_FL_NUM_indexer = StringIndexer(inputCol='OP_CARRIER_FL_NUM',outputCol='OP_CARRIER_FL_NUM_index',handleInvalid='keep')
ORIGIN_indexer = StringIndexer(inputCol='ORIGIN',outputCol='ORIGIN_index',handleInvalid='keep')
DEST_indexer = StringIndexer(inputCol='DEST',outputCol='DEST_index',handleInvalid='keep')
DEP_DEL15_indexer = StringIndexer(inputCol='DEP_DEL15',outputCol='DEP_DEL15_index',handleInvalid='keep')
DEP_DELAY_GROUP_indexer = StringIndexer(inputCol='DEP_DELAY_GROUP',outputCol='DEP_DELAY_GROUP_index',handleInvalid='keep')
CARRIER_DELAY_indexer = StringIndexer(inputCol='CARRIER_DELAY',outputCol='CARRIER_DELAY_index',handleInvalid='keep')
WEATHER_DELAY_indexer = StringIndexer(inputCol='WEATHER_DELAY',outputCol='WEATHER_DELAY_index',handleInvalid='keep')
NAS_DELAY_indexer = StringIndexer(inputCol='NAS_DELAY',outputCol='NAS_DELAY_index',handleInvalid='keep')
SECURITY_DELAY_indexer = StringIndexer(inputCol='SECURITY_DELAY',outputCol='SECURITY_DELAY_index',handleInvalid='keep')
LATE_AIRCRAFT_DELAY_indexer = StringIndexer(inputCol='LATE_AIRCRAFT_DELAY',outputCol='LATE_AIRCRAFT_DELAY_index',handleInvalid='keep')

In [102]:
assembler = VectorAssembler(inputCols=['OP_UNIQUE_CARRIER_index','OP_CARRIER_FL_NUM_index','ORIGIN_index','DEST_index',
                                       'DEP_DEL15_index','DEP_DELAY_GROUP_index','CARRIER_DELAY_index','WEATHER_DELAY_index',
                                       'NAS_DELAY_index','SECURITY_DELAY_index','LATE_AIRCRAFT_DELAY_index'],
                            outputCol="features")

In [103]:
pipe = Pipeline(stages=[OP_UNIQUE_CARRIER_indexer,OP_CARRIER_FL_NUM_indexer,ORIGIN_indexer,DEST_indexer,DEP_DEL15_indexer,DEP_DELAY_GROUP_indexer,
                        CARRIER_DELAY_indexer,WEATHER_DELAY_indexer,NAS_DELAY_indexer,SECURITY_DELAY_indexer,LATE_AIRCRAFT_DELAY_indexer,assembler])

A confusion matrix is a summary of prediction results on a classification problem. The number of correct and incorrect predictions are summarized with count values and broken down by each class. The best accuracy is 1.0, whereas the worst is 0.0. It can also be calculated by 1 – ERR. Accuracy is calculated as the total number of two correct predictions (TP + TN) divided by the total number of a dataset (P + N). Best accuracy when value are high diagnolly and low non-diagnolly.

Inference:
    Based on above two models, model can do classification based on departure delay of 15min or not with acuuracy,most of departures delays fall under the right category & the duration estimation is also predicted accurately.

NOTE:
-spark function are reqired to run on master nodes,running only python code might work in worker node but cannot assure in master node
-Though the above code looks like in python,running in this program might not guarnatee in bigdata environment as the data will increase

-------------------------------------------------------------------------------------------------------------------------------